In [1]:
import pandas as pd
import numpy as np
import joblib
import os 

In [2]:
SCALER = 'scaler.pkl'

MODELS_PATH = '../models/'
DATA_PATH = '../data/'

ENCODER_PATH = DATA_PATH + 'encoder/'
SCALER_PATH = DATA_PATH + 'Scaler/'

In [3]:
target_columns = ['Exercises_encoded', 'Equipment_encoded', 
                  'Diet_encoded', 'Recommendation_encoded']

In [4]:
scaler_features = joblib.load(SCALER_PATH + "scaler_features.pkl")

In [5]:
scaler = joblib.load(os.path.join(SCALER_PATH, SCALER))
models = {}

for col in target_columns:
    model_file = [f for f in os.listdir(MODELS_PATH) if f.startswith(col)][0]
    models[col] = joblib.load(os.path.join(MODELS_PATH, model_file))

In [6]:
def encode_value(col_name, val):
    path = os.path.join(ENCODER_PATH, f"{col_name.replace('_encoded','')}_encoder.csv")
    df = pd.read_csv(path)
    mapping = dict(zip(df['label'], df['encoded']))
    return mapping.get(val, np.nan)


In [7]:
def decode_value(col_name, val):
    path = os.path.join(ENCODER_PATH, f"{col_name.replace('_encoded','')}_encoder.csv")
    df = pd.read_csv(path)
    mapping = dict(zip(df['encoded'], df['label']))
    return mapping.get(val, np.nan)

In [8]:
def predict_user_input(user_input):
    input_values = {}
    for col, val in user_input.items():
        if isinstance(val, str) and col.endswith('_encoded'):
            val = encode_value(col, val)
        input_values[col] = val
    
    df_raw = pd.DataFrame([input_values])
    df_scaled = pd.DataFrame([input_values])
    df_scaled[scaler_features] = scaler.transform(df_scaled[scaler_features])
    
    predictions = {}
    for col, model in models.items():
        predictions[col] = model.predict(df_scaled)[0] 
    
    return predictions, df_raw, df_scaled

In [9]:
user_input = {
    'Age': 18,
    'Height': 168,
    'Weight': 47.5,
    'Sex_encoded': 'Male',
    'Hypertension_encoded': 'Yes',
    'Diabetes_encoded': 'No',
    'Fitness Goal_encoded': 'Weight Gain'
}

In [10]:
prediction, df_raw, df_ready = predict_user_input(user_input)

print("=== Hasil Prediksi (Before Decoded) ===")
display(prediction)

print("=== Hasil Prediksi (After Decoded) ===")
decoded_predictions = {col: decode_value(col, val) for col, val in prediction.items()}
display(decoded_predictions)

# print("\n=== Input User (Sebelum Scaling) ===")
# display(df_raw)
# print("\n=== Input User (Setelah Scaling) ===")
# display(df_ready)

=== Hasil Prediksi (Before Decoded) ===


{'Exercises_encoded': np.int64(1),
 'Equipment_encoded': np.int64(6),
 'Diet_encoded': np.int64(9),
 'Recommendation_encoded': np.int64(4)}

=== Hasil Prediksi (After Decoded) ===


{'Exercises_encoded': 'Squats, deadlifts, bench presses, and overhead presses',
 'Equipment_encoded': 'Light athletic shoes, resistance bands, and light dumbbells.',
 'Diet_encoded': 'Vegetables: (Tomatoes, Garlic, leafy greens, broccoli, carrots, and bell peppers); Protein Intake: (poultry, fish, tofu, legumes, and low-fat dairy products); Juice: (Apple juice, beetroot juice and mango juice)',
 'Recommendation_encoded': "Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. I'm not a medical professional, but I can give you general advice that you should talk to your healthcare provider before making any changes to your exercise or diet, especially if you have hypertension. It is important to work with a health care professional to create an individualized plan based on your unique health needs. Always consult a health care professional or dietician for personalized advice based on your specific medical condition, as high blood pressure requir